# Cleaning Data: fixing values

So far, we've dealt with structural issues in data. but there's a lot more to
cleaning.  

Today,  we'll deal with how to fix the values within  the data.

Examples of how values can be represented poorly:


[Stanford Policy Lab Open Policing Project data readme](https://github.com/stanford-policylab/opp/blob/master/data_readme.md)
[Propublica Machine Bias](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm) the "How we acquired data" section

```{hint}
 you can treat this one proejct as multiple cleaned datasets and study it more carefully for A4.
```

## Admin

````{margin}

```{admonition} Hacktoberfest

learn about Hacktoberfest](https://hacktoberfest.com/)
```
````

- [watch this repo](https://github.com/rhodyprog4ds/BrownFall22) to get announcements
- grading will be updated over the weekend!
- remember to [accept](https://classroom.github.com/a/-7TUoBDE) [a4](https://rhodyprog4ds.github.io/BrownFall22/assignments/04-prepare.html)


```{important}
Use the Submit workflow on your Actions tab for each assignment
```

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np #
na_toy_df = pd.DataFrame(data = [[1,3,4,5],[2 ,6, np.nan,4]])

# make plots look nicer and increase font size
sns.set_theme(font_scale=2)
arabica_data_url = 'https://raw.githubusercontent.com/jldbc/coffee-quality-database/master/data/arabica_data_cleaned.csv'

coffee_df = pd.read_csv(arabica_data_url)


rhodyprog4ds_gh_events_url = 'https://api.github.com/orgs/rhodyprog4ds/events'
course_gh_df = pd.read_json(rhodyprog4ds_gh_events_url)

## Missing Values


Dealing with missing data is a whole research area. There isn't one solution.


[in 2020 there was a whole workshop on missing](https://artemiss-workshop.github.io/)

one organizer is the main developer of [sci-kit learn](https://scikit-learn.org/stable/) the ML package we will use soon.  In a [2020 invited talk](https://static.sched.com/hosted_files/ray2020/08/Keynote-%20Easier%20Machine%20Learning%20Thoughts%20From%20Scikit-Learn%20-%20Ga%C3%ABl%20Varoquaux%2C%20Research%20Director%2C%20Inria.pdf) he listed more automatic handling as an active area of research  and a development goal for sklearn.

There are also many classic approaches both when training and when [applying models](https://www.jmlr.org/papers/volume8/saar-tsechansky07a/saar-tsechansky07a.pdf).


[example application in breast cancer detection](https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.701.4234&rep=rep1&type=pdf)


Even in pandas, dealing with [missing values](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html) is under [experimentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html#missing-data-na)
 as to how to represent it symbolically


Missing values even causes the [datatypes to change](https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html#missing-data-casting-rules-and-indexing)


That said, there are are om
Pandas gives a few basic tools:

- dropna
- fillna



Dropping is a good choice when you otherwise have a lot of data and the data is
missing at random.

Dropping can be risky if it's not missing at random. For example, if we saw in
the coffee data that one of the scores was missing for all of the rows from one
country, or even just missing more often in one country, that could bias our
results.  

Filling can be good if you know how to fill reasonably, but don't have data to
spare by dropping.  For example
- you can approximate with another column
- you can approximate with that column from other rows


**whatever you do, document it**

### Finding Missing Values

Our skill in summarixing and getting overviews of data helps us know when we
have a problem.

In [2]:
coffee_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1311 entries, 0 to 1310
Data columns (total 44 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             1311 non-null   int64  
 1   Species                1311 non-null   object 
 2   Owner                  1304 non-null   object 
 3   Country.of.Origin      1310 non-null   object 
 4   Farm.Name              955 non-null    object 
 5   Lot.Number             270 non-null    object 
 6   Mill                   1001 non-null   object 
 7   ICO.Number             1165 non-null   object 
 8   Company                1102 non-null   object 
 9   Altitude               1088 non-null   object 
 10  Region                 1254 non-null   object 
 11  Producer               1081 non-null   object 
 12  Number.of.Bags         1311 non-null   int64  
 13  Bag.Weight             1311 non-null   object 
 14  In.Country.Partner     1311 non-null   object 
 15  Harv

### Example Filling

The 'Lot.Number' has a lot of NaN values, how can we explore it?

We can look at the type:

In [3]:
coffee_df['Lot.Number'].dtype

dtype('O')

And we can look at the value counts.

In [4]:
coffee_df['Lot.Number'].value_counts()

1                             18
020/17                         6
019/17                         5
2                              3
102                            3
                              ..
11/23/0696                     1
3-59-2318                      1
8885                           1
5055                           1
017-053-0211/ 017-053-0212     1
Name: Lot.Number, Length: 221, dtype: int64

We see that a lot are '1', maybe we know that when the data was collected, if the Farm only has one lot, some people recorded '1' and others left it as missing. So we could fill in with 1:

In [5]:
coffee_df['Lot.Number'].fillna(1).head(10)

0    1
1    1
2    1
3    1
4    1
5    1
6    1
7    1
8    1
9    1
Name: Lot.Number, dtype: object

Note that even after we called `fillna` we display it again and the original data is unchanged.

In [6]:
coffee_df['Lot.Number'].head(3)

0    NaN
1    NaN
2    NaN
Name: Lot.Number, dtype: object

To save the filled in column we have a few choices:
- use the `inplace` parameter. This doesn't offer performance advantages, but does It still copies the object, but then reassigns the pointer. Its under discussion to [deprecate](https://github.com/pandas-dev/pandas/issues/16529)
- write to a new DataFrame
- add a column


We'll use adding a column:

In [7]:
coffee_df['lot_number_clean'] = coffee_df['Lot.Number'].fillna('1')

In [8]:
coffee_df['lot_number_clean'].value_counts()

1                             1059
020/17                           6
019/17                           5
102                              3
103                              3
                              ... 
3-59-2318                        1
8885                             1
5055                             1
MCCFWXA15/16                     1
017-053-0211/ 017-053-0212       1
Name: lot_number_clean, Length: 221, dtype: int64

### Example Dropping



To illustrate how `dropna` works, we'll use the `shape` method after each call.  
Dropna is going to drop either rows or columns. So we

In [9]:
coffee_df.shape

(1311, 45)

By default, it drops any row with one or more `NaN` values.

In [10]:
coffee_df.dropna().shape

(130, 45)

We could instead tell it to only drop rows with `NaN` in a subset of the columns.
We might do this if we were interested in studying the impact of altitude on the ratings .

In [11]:
coffee_df.dropna(subset=['altitude_low_meters']).shape

(1084, 45)

In the [Open Policing Project Data Summary](https://openpolicing.stanford.edu/data/) we saw that they made a summary information that showed which variables had at least 70% not missing values.  We can similarly choose to keep only variables that have more than a specific threshold of data, using the `thresh` parameter and `axis=1` to drop along columns.

The `thresh` parameter requires an `int` not a fraction of the rows, so we can
save the shape values to variables and then use that

In [12]:
nrows, ncols = coffee_df.shape
coffee_df.dropna(thresh =.7*nrows,axis=1).shape

(1311, 44)

This dataset is actually in pretty good shape, but if we use a more stringent threshold it drops more columns.

In [13]:
nrows, ncols = coffee_df.shape
coffee_df.dropna(thresh =.85*nrows,axis=1).shape

(1311, 34)

## Inconsistent values

This was one of the things that many of you anticipated or had observed.  A useful way to investigate for this, is to use `value_counts` and sort them alphabetically by the values from the original data, so that similar ones will be consecutive in the list. Once we have the `value_counts()` Series, the values from the `coffee_df` become the index, so we use `sort_index`.

Let's look at the `In.Country.Partner` column
```
coffee_df['In.Country.Partner'].value_counts().sort_index()
```
---
We can see there's only one `Blossom Valley International\n` but 58 `Blossom Valley International`, the former is likely a typo, especially since `\n` is a special character for a newline. Similarly, with 'Specialty Coffee Ass' and 'Specialty Coffee Association'.
---
This is another job for dictionaries, we make one with the value to replace as the key and the value to insert as the value.
```
partner_corrections = {'Blossom Valley International\n':'Blossom Valley International',
  'Specialty Coffee Ass':'Specialty Coffee Association'}
coffee_df['in_country_partner_clean'] = coffee_df['In.Country.Partner'].replace(
  to_replace=partner_corrections)
coffee_df['in_country_partner_clean'].value_counts().sort_index()
```


## Fixing data at load time

Some of the different parameters in `read_csv` can also fix how it reads in data.

For example `header` can make somethign like this:

![mulitindex img of excel file](https://github.com/rhodyprog4ds/BrownFall20/raw/main/img/multiindex.png)

read in correctly.  



## A Cleaning Data Recipe

__not everything possible, but good enough for this course__


1. Can you use parameters to read the data in better?
1. Fix the index and column headers (making these easier to use makes the rest easier)
1. Is the data strucutred well?
1. Are there missing values?
1. Do the datatypes match what you expect by looking at the head or a sample?
1. Are categorical variables represented in usable way?
1. Does your analysis require filtering or augmenting the data?




## Further reading



Instead of more practice with these manipulations, below are more
examples of cleaning data to see how these types of manipulations get used.  
Your goal here is not to memorize every possible thing, but to build a general
idea of what good data looks like and good habits for cleaning data and keeping
it reproducible.  
- [Cleaning the Adult Dataset](https://ryanwingate.com/projects/machine-learning-data-prep/adult/adult-cleaning/)
- [All Shades](https://github.com/the-pudding/data/tree/master/foundation-names#allshadescsv--allshadesr)

Also here are some tips on general data management and organization.

This article is a comprehensive [discussion of data cleaning](https://towardsdatascience.com/the-ultimate-guide-to-data-cleaning-3969843991d4).


##  Questions 

```{important}
I will add these later. I have a deadline tomorrow.
```